### RAG (Retrieval-Augmented Generation)
> 학습된 지식 + 실시간으로 외부 데이터를 검색하고 활용해 더 정확한 답변 생성

 기존의 대규모 언어 모델(LLM)을 확장하여, 주어진 질문에 대해 좀 더 정확하고 풍부한 답변을 제공하기 위한 방법입니다. 이 기법은 검색(Retrieval)과 생성(Generation) 과정을 결합하여, 모델이 학습된 지식만을 바탕으로 답변을 생성하는 것이 아니라, 실시간으로 외부 데이터를 검색하여 그 정보를 활용하여 더 정확한 답변을 생성

 - **검색 단계 (Retrieval)**
    
    사용자의 질문이나 컨텍스트에 맞는 외부 데이터를 실시간으로 검색합니다. 이 데이터는 모델이 답변을 생성하는 데 필요한 **자세하고 정확한 정보**를 포함하고 있어야 하며, 여러 문서 중 적합한 정보를 선택하는 과정이 포함됩니다.
    
    **주요 구성 요소:**
    
    - **문서 인덱싱**: 외부 데이터베이스나 문서를 검색 가능한 형태로 변환
    - **임베딩 생성**: 문서와 질문을 벡터 공간에서 표현
    - **유사도 계산**: 질문과 관련성이 높은 문서를 찾기 위한 유사도 측정
    - **상위 K개 문서 선택**: 가장 관련성이 높은 문서들을 선별
- **생성 단계 (Generation)**
    
    검색된 정보를 기반으로 LLM이 사용자의 질문에 대해 자연스럽고 정확한 답변을 생성합니다. 이 과정에서 모델은 **검색된 데이터와 기존의 지식을 결합**하여 답변을 만들며, 생성된 답변은 사용자가 필요로 하는 정보와 맥락을 고려하여 정확히 맞추어집니다.

- 장점: 실시간 정보 반영, 정확한 답변, 할루시네이션 방지
- 단점: 검색 품질 의존성, 지연 시간, 복잡성 증가, 컨텍스트 길이 제한

### RAG의 주요 구현 기술

- 벡터 데이터베이스
    - **Pinecone** : 관리형 벡터 데이터베이스 서비스 ( https://www.pinecone.io/ )
    - **Weaviate** : 오픈소스 벡터 데이터베이스 ( https://weaviate.io/ )
    - **Chroma** : 경량화 된 벡터 데이터베이스 ( https://www.trychroma.com/ )
    - **FAISS** : Facebook이 개발한 유사도 검색 라이브러리 ( https://faiss.ai/ )
- 임베딩 모델
    - **Sentence-BERT** : 문장 임베딩 특화 모델 ( https://sbert.net/ )
    - **OpenAI Embeddings** : GPT 기반 임베딩 모델
        - https://platform.openai.com/docs/guides/embeddings
    - **E5** : Microsoft의 다국어 임베딩 모델
        - https://huggingface.co/intfloat/multilingual-e5-large
    - **BGE** : [중국 BAAI](https://en.wikipedia.org/wiki/Beijing_Academy_of_Artificial_Intelligence)에서 개발한 고성능 임베딩 모델
        - https://huggingface.co/BAAI/bge-m3
- 검색 방법론
    - **Dense Retrieval** : 벡터 기반 유사도 검색
    - **Sparse Retrieval** : 키워드 기반 검색 (BM25 등)
    - **Hybrid Retrieval** : Dense와 Sparse 방법의 결합

#### RAG 파이프라인
- Data Sources → Load Data → Text Split → Indexing → Query Processing → Retrieval → Generation → Post-processing